In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from model import build_unet
from metrics import dice_loss, dice_coef,iou

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
""" Global parameters """
H = 512
W = 512

def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "CXR_png", "*.png")))
    masks1 = sorted(glob(os.path.join(path, "masks", "*.png")))
    

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y1, valid_y1 = train_test_split(masks1, test_size=split_size, random_state=42)
  

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y1, test_y1 = train_test_split(train_y1, test_size=split_size, random_state=42)
    

    return (train_x, train_y1), (valid_x, valid_y1), (test_x, test_y1)

In [ ]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [ ]:
def read_mask(path1):
    x = cv2.imread(path1, cv2.IMREAD_GRAYSCALE)
   
    x = cv2.resize(x, (W, H))
    x = x/np.max(x)
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [ ]:
def tf_parse(x, y1):
    def _parse(x, y1):
        x = x.decode()
        y1 = y1.decode()
       

        x = read_image(x)
        y = read_mask(y1)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y1], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [ ]:
def tf_dataset(X, Y1, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y1))
    dataset = dataset.shuffle(buffer_size=200)
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 2
    lr = 1e-5
    num_epochs = 5
    model_path = os.path.join("files", "model.h5")
    csv_path = os.path.join("files", "data.csv")

    """ Dataset """
    dataset_path = "/content/gdrive/My Drive/Covid"
    (train_x, train_y1), (valid_x, valid_y1), (test_x, test_y1) = load_data(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y1)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y1)}")
    print(f"Test: {len(test_x)} - {len(test_y1)}")

    train_dataset = tf_dataset(train_x, train_y1, batch=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y1, batch=batch_size)

    """ Model """
    model = build_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Train: 80 - 80
Valid: 10 - 10
Test: 10 - 10
Epoch 1/5
40/40 [==============================] - ETA: 0s - loss: 0.5973 - dice_coef: 0.4027 - iou: 0.2549 - recall: 0.5716 - precision: 0.3980 
Epoch 1: val_loss improved from inf to 0.64490, saving model to files/model.h5
40/40 [==============================] - 2167s 54s/step - loss: 0.5973 - dice_coef: 0.4027 - iou: 0.2549 - recall: 0.5716 - precision: 0.3980 - val_loss: 0.6449 - val_dice_coef: 0.3551 - val_iou: 0.2162 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - lr: 1.0000e-05
Epoch 2/5
40/40 [==============================] - ETA: 0s - loss: 0.4558 - dice_coef: 0.5442 - iou: 0.3759 - recall: 0.8477 - precision: 0.5096 
Epoch 2: val_loss did not improve from 0.64490
40/40 [==============================] - 2169s 54s/step - loss: 0.4558 - dice_coef: 0.5442 - iou: 0.3759 - recall: 0.8477 - precision: 0.5096 - val_loss: 0.6483 - val_dice_coef: 0.3517 - val_iou: 0.2141 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - lr: 1.0

In [ ]:
print("iou", history.history['iou'][-1])
print("dice", history.history['dice_coef'][-1])

NameError: ignored